In [9]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(1_000_000_000_000) ## 1TB?

from calitp.tables import tbl
from calitp import query_sql

import pandas as pd
import geopandas as gpd
from siuba import *

pd.set_option('display.max_columns', None) 
import shared_utils

# NHTS Stats
Pulling : mode share of transit / all trips, types of transit within transit

In [2]:
GCS_FILE_PATH = 'gs://calitp-analytics-data/data-analyses/nhts_data/'

## 2020 Origin-Destination Data

In [3]:
nhts_all = pd.read_csv(f'{GCS_FILE_PATH}National Passenger Final OD Data_2020.csv')

nhts_all.head(5)

,year,origin_zone_id,destination_zone_id,origin_zone_name,destination_zone_name,origin_state,destination_state,annual_total_trips,mode_air,mode_rail,...,vehicle_150_300mi_pct,vehicle_gt300mi_pct,atf_0_10mi_pct,atf_10_25mi_pct,atf_25_50mi_pct,atf_50_75mi_pct,atf_75_100mi_pct,atf_100_150mi_pct,atf_150_300mi_pct,atf_gt300mi_pct
0,2020,10180_TX,10180_TX,"Abilene, TX","Abilene, TX",TX,TX,179881350,0,0,...,0.0,0.0,0.994248,0.004383,0.001369,0.0,0.0,0.0,0.0,0.0
1,2020,10420_OH,10180_TX,"Akron, OH","Abilene, TX",OH,TX,43,17,0,...,0.0,1.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
2,2020,10500_GA,10180_TX,"Albany, GA","Abilene, TX",GA,TX,0,0,0,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
3,2020,10540_OR,10180_TX,"Albany, OR","Abilene, TX",OR,TX,0,0,0,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
4,2020,10580_NY,10180_TX,"Albany-Schenectady-Troy, NY","Abilene, TX",NY,TX,120,57,0,...,0.0,1.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0


In [6]:
# filter to CA trips
nhts_ca = (nhts_all
           >> filter(_.origin_state=="CA",_.destination_state=="CA")
          )

nhts_ca.head(5)

,year,origin_zone_id,destination_zone_id,origin_zone_name,destination_zone_name,origin_state,destination_state,annual_total_trips,mode_air,mode_rail,mode_vehicle,mode_atf,purpose_work,purpose_nonwork,distance_0_10,distance_10_25,distance_25_50,distance_50_75,distance_75_100,distance_100_150,distance_150_300,distance_gt300,work_0_10mi,work_10_25mi,work_25_50mi,work_50_75mi,work_75_100mi,work_100_150mi,work_150_300mi,work_gt300mi,nonwork_0_10mi,nonwork_10_25mi,nonwork_25_50mi,nonwork_50_75mi,nonwork_75_100mi,nonwork_100_150mi,nonwork_150_300mi,nonwork_gt300mi,air_0_10mi,air_10_25mi,air_25_50mi,air_50_75mi,air_75_100mi,air_100_150mi,air_150_300mi,air_gt300mi,rail_0_10mi,rail_10_25mi,rail_25_50mi,rail_50_75mi,rail_75_100mi,rail_100_150mi,rail_150_300mi,rail_gt300mi,vehicle_0_10mi,vehicle_10_25mi,vehicle_25_50mi,vehicle_50_75mi,vehicle_75_100mi,vehicle_100_150mi,vehicle_150_300mi,vehicle_gt300mi,atf_0_10mi,atf_10_25mi,atf_25_50mi,atf_50_75mi,atf_75_100mi,atf_100_150mi,atf_150_300mi,atf_gt300mi,mode_air_pct,mode_rail_pct,mode_vehicle_pct,mode_atf_pct,purpose_work_pct,purpose_nonwork_pct,distance_0_10_pct,distance_10_25_pct,distance_25_50_pct,distance_50_75_pct,distance_75_100_pct,distance_100_150_pct,distance_150_300_pct,distance_gt300_pct,work_0_10mi_pct,work_10_25mi_pct,work_25_50mi_pct,work_50_75mi_pct,work_75_100mi_pct,work_100_150mi_pct,work_150_300mi_pct,work_gt300mi_pct,nonwork_0_10mi_pct,nonwork_10_25mi_pct,nonwork_25_50mi_pct,nonwork_50_75mi_pct,nonwork_75_100mi_pct,nonwork_100_150mi_pct,nonwork_150_300mi_pct,nonwork_gt300mi_pct,air_0_10mi_pct,air_10_25mi_pct,air_25_50mi_pct,air_50_75mi_pct,air_75_100mi_pct,air_100_150mi_pct,air_150_300mi_pct,air_gt300mi_pct,rail_0_10mi_pct,rail_10_25mi_pct,rail_25_50mi_pct,rail_50_75mi_pct,rail_75_100mi_pct,rail_100_150mi_pct,rail_150_300mi_pct,rail_gt300mi_pct,vehicle_0_10mi_pct,vehicle_10_25mi_pct,vehicle_25_50mi_pct,vehicle_50_75mi_pct,vehicle_75_100mi_pct,vehicle_100_150mi_pct,vehicle_150_300mi_pct,vehicle_gt300mi_pct,atf_0_10mi_pct,atf_10_25mi_pct,atf_25_50mi_pct,atf_50_75mi_pct,atf_75_100mi_pct,atf_100_150mi_pct,atf_150_300mi_pct,atf_gt300mi_pct
14016,2020,12540_CA,12540_CA,"Bakersfield, CA","Bakersfield, CA",CA,CA,836876424,0,10856,736724686,100140882,144282182,692594242,724806378,79007936,25170342,6085200,1265524,531748,9290,6,121830256,15567562,4993274,1302644,422158,160740,5544,4,602976122,63440374,20177068,4782556,843366,371008,3746,2,0,0,0,0,0,0,0,0,6142,2888,1402,134,14,0,270,6,625637598,78211116,24984628,6085066,1265510,531748,9020,0,99162638,793932,184312,0,0,0,0,0,0.000000,0.000013,0.880327,0.119660,0.172406,0.827594,0.866085,0.094408,0.030077,0.007271,0.001512,0.000635,0.000011,7.169517e-09,0.844389,0.107897,0.034608,0.009028,0.002926,0.001114,0.000038,2.772345e-08,0.870605,0.091598,0.029133,0.006905,0.001218,0.000536,0.000005,2.887694e-09,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.56577,0.266028,0.129145,0.012343,0.001290,0.000000,0.024871,0.000553,0.849215,0.106161,0.033913,0.008260,0.001718,0.000722,0.000012,0.000000,0.990231,0.007928,0.001841,0.0,0.0,0.0,0.0,0.0
14069,2020,17020_CA,12540_CA,"Chico, CA","Bakersfield, CA",CA,CA,8423,46,1,8376,0,1216,7207,0,0,0,0,0,0,0,8423,0,0,0,0,0,0,0,1216,0,0,0,0,0,0,0,7207,0,0,0,0,0,0,0,46,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,8376,0,0,0,0,0,0,0,0,0.005461,0.000119,0.994420,0.000000,0.144367,0.855633,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000e+00,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,1.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
14113,2020,20940_CA,12540_CA,"El Centro, CA","Bakersfield, CA",CA,CA,31991,17,0,31974,0,12292,19699,0,0,0,0,0,0,11551,20440,0,0,0,0,0,0,4816,7476,0,0,0,0,0,0,6735,12964,0,0,0,0,0,0,17,0,0,0,0,

In [11]:
# summary of air/rail/veh/"atf"

nhts_ca_sum = (nhts_ca 
>> group_by(_.year)
>> summarize(sum_total_trips=_.annual_total_trips.sum(),
             sum_mode_air=_.mode_air.sum(),
             sum_mode_rail=_.mode_rail.sum(),
             sum_mode_vehicle=_.mode_vehicle.sum(),
             sum_mode_atf=_.mode_atf.sum()
            )
>> ungroup()
>> mutate(pct_vehicle = _.sum_mode_vehicle/_.sum_total_trips,
          pct_rail = _.sum_mode_rail/_.sum_total_trips,
          pct_atf = _.sum_mode_atf/_.sum_total_trips
         )
              )
             

In [12]:
nhts_ca_sum

,year,sum_total_trips,sum_mode_air,sum_mode_rail,sum_mode_vehicle,sum_mode_atf,pct_vehicle,pct_rail,pct_atf
0,2020,35476104716,7112430,133735502,30621807188,4713449596,0.863167,0.00377,0.132863


## 2017 Full NHTS

In [14]:
trips_all = pd.read_csv(f'{GCS_FILE_PATH}trippub.csv')

trips_all.head(5)

,HOUSEID,PERSONID,TDTRPNUM,STRTTIME,ENDTIME,TRVLCMIN,TRPMILES,TRPTRANS,TRPACCMP,TRPHHACC,VEHID,TRWAITTM,NUMTRANS,TRACCTM,DROP_PRK,TREGRTM,WHODROVE,WHYFROM,LOOP_TRIP,TRPHHVEH,HHMEMDRV,HH_ONTD,NONHHCNT,NUMONTRP,PSGR_FLG,PUBTRANS,TRIPPURP,DWELTIME,TDWKND,VMT_MILE,DRVR_FLG,WHYTRP1S,ONTD_P1,ONTD_P2,ONTD_P3,ONTD_P4,ONTD_P5,ONTD_P6,ONTD_P7,ONTD_P8,ONTD_P9,ONTD_P10,ONTD_P11,ONTD_P12,ONTD_P13,TDCASEID,TRACC_WLK,TRACC_POV,TRACC_BUS,TRACC_CRL,TRACC_SUB,TRACC_OTH,TREGR_WLK,TREGR_POV,TREGR_BUS,TREGR_CRL,TREGR_SUB,TREGR_OTH,WHYTO,TRAVDAY,HOMEOWN,HHSIZE,HHVEHCNT,HHFAMINC,DRVRCNT,HHSTATE,HHSTFIPS,NUMADLT,WRKCOUNT,TDAYDATE,HHRESP,LIF_CYC,MSACAT,MSASIZE,RAIL,URBAN,URBANSIZE,URBRUR,GASPRICE,CENSUS_D,CENSUS_R,CDIVMSAR,HH_RACE,HH_HISP,HH_CBSA,SMPLSRCE,R_AGE,EDUC,R_SEX,PRMACT,PROXY,WORKER,DRIVER,WTTRDFIN,WHYTRP90,TRPMILAD,R_AGE_IMP,R_SEX_IMP,VEHTYPE,OBHUR,DBHUR,OTHTNRNT,OTPPOPDN,OTRESDN,OTEEMPDN,OBHTNRNT,OBPPOPDN,OBRESDN,DTHTNRNT,DTPPOPDN,DTRESDN,DTEEMPDN,DBHTNRNT,DBPPOPDN,DBRESDN
0,30000007,1,1,1000,1015,15,5.244,3,0,0,3,-1,-1,-1,-1,-1,1,1,2,1,1,1,0,1,2,2,HBO,295,2,5.244,1,20,1,2,2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,300000070101,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,19,2,1,3,5,7,3,NC,37,3,1,201608,1,10,3,1,2,1,1,1,228.4,5,3,53,2,2,XXXXX,2,67,3,2,6,1,2,1,75441.905796,5,5.847584,67,2,1,T,T,50,1500,750,750,20,750,300,50,750,300,350,30,300,300
1,30000007,1,2,1510,1530,20,5.149,3,0,0,3,-1,-1,-1,-1,-1,1,19,2,1,1,1,0,1,2,2,HBO,-9,2,5.149,1,1,1,2,2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,300000070102,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,2,1,3,5,7,3,NC,37,3,1,201608,1,10,3,1,2,1,1,1,228.4,5,3,53,2,2,XXXXX,2,67,3,2,6,1,2,1,75441.905796,5,5.741650,67,2,1,T,T,50,750,300,350,30,300,300,50,1500,750,750,20,750,300
2,30000007,2,1,700,900,120,84.004,6,0,0,5,-1,-1,-1,-1,-1,2,3,2,1,1,1,0,1,2,2,HBW,540,2,84.004,1,1,2,1,2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,300000070201,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,2,1,3,5,7,3,NC,37,3,1,201608,1,10,3,1,2,1,1,1,228.4,5,3,53,2,2,XXXXX,2,66,3,1,1,2,1,1,71932.645806,1,90.178294,66,1,4,S,T,40,1500,750,750,40,1500,750,50,1500,750,750,20,750,300
3,30000007,2,2,1800,2030,150,81.628,6,0,0,5,-1,-1,-1,-1,-1,2,1,2,1,1,1,0,1,2,2,HBW,-9,2,81.628,1,10,2,1,2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,300000070202,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,3,2,1,3,5,7,3,NC,37,3,1,201608,1,10,3,1,2,1,1,1,228.4,5,3,53,2,2,XXXXX,2,66,3,1,1,2,1,1,71932.645806,1,87.627658,66,1,4,T,S,50,1500,750,750,20,750,300,40,1500,750,750,40,1500,750
4,30000007,3,1,845,900,15,2.250,3,0,0,1,-1,-1,-1,-1,-1,3,1,2,1,1,1,0,1,2,2,HBO,330,2,2.250,1,20,2,2,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,300000070301,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,8,2,1,3,5,7,3,NC,37,3,1,201608,1,10,3,1,2,1,1,1,228.4,5,3,53,2,2,XXXXX,2,28,2,2,5,2,2,1,80122.686739,5,2.508975,28,2,1,T,T,50,1500,750,750,20,750,300,50,750,300,350,60,750,300


In [15]:
trips_ca = (trips_all
            >> filter(_.HHSTATE=="CA")
           )

In [17]:
# add person weights
perwgt=pd.read_csv(f'{GCS_FILE_PATH}perwgt.csv')

perwgt.head(5)

,HOUSEID,PERSONID,WTPERFIN,WTPERFIN1,WTPERFIN2,WTPERFIN3,WTPERFIN4,WTPERFIN5,WTPERFIN6,WTPERFIN7,WTPERFIN8,WTPERFIN9,WTPERFIN10,WTPERFIN11,WTPERFIN12,WTPERFIN13,WTPERFIN14,WTPERFIN15,WTPERFIN16,WTPERFIN17,WTPERFIN18,WTPERFIN19,WTPERFIN20,WTPERFIN21,WTPERFIN22,WTPERFIN23,WTPERFIN24,WTPERFIN25,WTPERFIN26,WTPERFIN27,WTPERFIN28,WTPERFIN29,WTPERFIN30,WTPERFIN31,WTPERFIN32,WTPERFIN33,WTPERFIN34,WTPERFIN35,WTPERFIN36,WTPERFIN37,WTPERFIN38,WTPERFIN39,WTPERFIN40,WTPERFIN41,WTPERFIN42,WTPERFIN43,WTPERFIN44,WTPERFIN45,WTPERFIN46,WTPERFIN47,WTPERFIN48,WTPERFIN49,WTPERFIN50,WTPERFIN51,WTPERFIN52,WTPERFIN53,WTPERFIN54,WTPERFIN55,WTPERFIN56,WTPERFIN57,WTPERFIN58,WTPERFIN59,WTPERFIN60,WTPERFIN61,WTPERFIN62,WTPERFIN63,WTPERFIN64,WTPERFIN65,WTPERFIN66,WTPERFIN67,WTPERFIN68,WTPERFIN69,WTPERFIN70,WTPERFIN71,WTPERFIN72,WTPERFIN73,WTPERFIN74,WTPERFIN75,WTPERFIN76,WTPERFIN77,WTPERFIN78,WTPERFIN79,WTPERFIN80,WTPERFIN81,WTPERFIN82,WTPERFIN83,WTPERFIN84,WTPERFIN85,WTPERFIN86,WTPERFIN87,WTPERFIN88,WTPERFIN89,WTPERFIN90,WTPERFIN91,WTPERFIN92,WTPERFIN93,WTPERFIN94,WTPERFIN95,WTPERFIN96,WTPERFIN97,WTPERFIN98,WTTRDFIN,WTTRDFIN1,WTTRDFIN2,WTTRDFIN3,WTTRDFIN4,WTTRDFIN5,WTTRDFIN6,WTTRDFIN7,WTTRDFIN8,WTTRDFIN9,WTTRDFIN10,WTTRDFIN11,WTTRDFIN12,WTTRDFIN13,WTTRDFIN14,WTTRDFIN15,WTTRDFIN16,WTTRDFIN17,WTTRDFIN18,WTTRDFIN19,WTTRDFIN20,WTTRDFIN21,WTTRDFIN22,WTTRDFIN23,WTTRDFIN24,WTTRDFIN25,WTTRDFIN26,WTTRDFIN27,WTTRDFIN28,WTTRDFIN29,WTTRDFIN30,WTTRDFIN31,WTTRDFIN32,WTTRDFIN33,WTTRDFIN34,WTTRDFIN35,WTTRDFIN36,WTTRDFIN37,WTTRDFIN38,WTTRDFIN39,WTTRDFIN40,WTTRDFIN41,WTTRDFIN42,WTTRDFIN43,WTTRDFIN44,WTTRDFIN45,WTTRDFIN46,WTTRDFIN47,WTTRDFIN48,WTTRDFIN49,WTTRDFIN50,WTTRDFIN51,WTTRDFIN52,WTTRDFIN53,WTTRDFIN54,WTTRDFIN55,WTTRDFIN56,WTTRDFIN57,WTTRDFIN58,WTTRDFIN59,WTTRDFIN60,WTTRDFIN61,WTTRDFIN62,WTTRDFIN63,WTTRDFIN64,WTTRDFIN65,WTTRDFIN66,WTTRDFIN67,WTTRDFIN68,WTTRDFIN69,WTTRDFIN70,WTTRDFIN71,WTTRDFIN72,WTTRDFIN73,WTTRDFIN74,WTTRDFIN75,WTTRDFIN76,WTTRDFIN77,WTTRDFIN78,WTTRDFIN79,WTTRDFIN80,WTTRDFIN81,WTTRDFIN82,WTTRDFIN83,WTTRDFIN84,WTTRDFIN85,WTTRDFIN86,WTTRDFIN87,WTTRDFIN88,WTTRDFIN89,WTTRDFIN90,WTTRDFIN91,WTTRDFIN92,WTTRDFIN93,WTTRDFIN94,WTTRDFIN95,WTTRDFIN96,WTTRDFIN97,WTTRDFIN98
0,30000007,1,206.690153,205.356568,213.999246,202.224399,203.510158,207.822681,206.364844,206.793195,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,232.962916,236.347584,242.961594,254.637297,240.077860,0.000000,246.003094,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,206.690177,75441.905796,74955.147339,78109.724891,73811.905783,74281.207689,75855.278607,75323.168110,75479.516089,75441.914489,75441.914489,75441.914489,75441.914489,75441.914489,75441.914489,75441.914489,75441.914489,75441.914489,75441.914489,75441.914489,75441.914489,75441.914489,75441.914489,75441.914489,75441.914489,75441.914489,75441.914489,75441.914489,75441.914489,75441.914489,75441.914489,75441.914489,75441.914489,75441.914489,75441.914489,75441.914489,75441.914489,75441.914489,75441.914489,75441.914489,75441.914489,75441.914489,75441.914489,75441.914489,75441.914489,75441.914489,75441.914489,75441.914489,75441.914489,75441.914489,75441.914489,75441.914489,75441.914489,75441.914489,75441.914489,75441.914489,75441.91448

In [18]:
perwgt = (perwgt
          >> select(_.HOUSEID,_.PERSONID,_.WTPERFIN)
          >> mutate(tripwt=_.WTPERFIN*365)
         )

perwgt.head(5)

,HOUSEID,PERSONID,WTPERFIN,tripwt
0,30000007,1,206.690153,75441.905796
1,30000007,2,197.075742,71932.645806
2,30000007,3,219.514210,80122.686739
3,30000008,1,63.185911,23062.857428
4,30000008,2,58.665911,21413.057483


In [26]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

trips_ca_sum = (trips_ca 
                >> left_join(_,perwgt)
                >> group_by(_.PUBTRANS,_.TRPTRANS)
                >> summarize(sum_trips=_.tripwt.sum(),
                            sum_vmt=(_.TRPMILES*_.tripwt).sum())
              )

trips_ca_sum

,PUBTRANS,TRPTRANS,sum_trips,sum_vmt
0,1,11,818659623.46243,6238894894.21688
1,1,15,101285410.39862,3482299607.12428
2,1,16,303427108.65505,4466558162.74979
3,2,-9,108328.73344,38023.38544
4,2,-8,312653.34153,423366.84103
5,2,1,5523696563.48595,3738346495.06671
6,2,2,550867124.11863,1160021508.16180
7,2,3,20838022268.86813,205347857043.89792
8,2,4,8497594528.83453,80070274186.80130
9,2,5,3085540383.46018,22950765492.20581


In [27]:
(trips_ca_sum
>> group_by(_.PUBTRANS)
>> summarize(sum_trips_pt=_.sum_trips.sum(),
            sum_vmt_pt=_.sum_vmt.sum())
 >> ungroup()
 >> mutate(pct=_.sum_trips_pt/_.sum_trips_pt.sum(),
           pct_vmt=_.sum_vmt_pt/_.sum_vmt_pt.sum())
)

,PUBTRANS,sum_trips_pt,sum_vmt_pt,pct,pct_vmt
0,1,1223372142.51610,14187752664.09094,0.02780,0.02966
1,2,42777567025.13925,464167504661.59894,0.97220,0.97034


In [28]:
(trips_ca_sum
>> group_by(_.PUBTRANS)
>> mutate(pct=_.sum_trips/_.sum_trips.sum(),
          pct_vmt=_.sum_vmt/_.sum_vmt.sum()
         )
)

,PUBTRANS,TRPTRANS,sum_trips,sum_vmt,pct,pct_vmt
0,1,11,818659623.46243,6238894894.21688,0.66918,0.43974
1,1,15,101285410.39862,3482299607.12428,0.08279,0.24544
2,1,16,303427108.65505,4466558162.74979,0.24803,0.31482
3,2,-9,108328.73344,38023.38544,0.00000,0.00000
4,2,-8,312653.34153,423366.84103,0.00001,0.00000
5,2,1,5523696563.48595,3738346495.06671,0.12913,0.00805
6,2,2,550867124.11863,1160021508.16180,0.01288,0.00250
7,2,3,20838022268.86813,205347857043.89792,0.48712,0.44240
8,2,4,8497594528.83453,80070274186.80130,0.19865,0.17250
9,2,5,3085540383.46018,22950765492.20581,0.07213,0.04945
